In [ ]:
# Coded By Sayan Chandra
# Roll : CS20M057
# DEEP LEARNING (CS6910) ASSIGNMENT 3
# Instructor : Mitesh M. Khapra  

In [ ]:
def unzip():
    !tar -xf dakshina_dataset_v1.0.tar

In [ ]:
def download():
    !wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

In [ ]:
def download_unzip():
    download()
    unzip()
download_unzip()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
def eid_did_dtd(qwerty):
  did = np.zeros((len(qwerty), max_decoder_seq_length), dtype="float32") #2D  
  dtd = np.zeros((len(qwerty), max_decoder_seq_length, num_decoder_tokens), dtype="float32") #3D
  eid = np.zeros((len(qwerty), max_encoder_seq_length), dtype="float32") #2D
  return eid, did, dtd

In [ ]:
def bookKeep(qwerty):
    print("global vars are set for training the model.")
    print("Num_samples:", len(qwerty))
    print("# unique input tokens:", num_encoder_tokens) # unique chars in english
    print("# unique output tokens:", num_decoder_tokens) # unique chars in hindi
    print("Max sequence length for inputs:", max_encoder_seq_length) # max wordlen in english
    print("Max sequence length for outputs:", max_decoder_seq_length) # max wordlen in hindi

In [ ]:
train_data_path = "dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv"
val_data_path =   "dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv"
test_data_path = "dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv"

In [ ]:
# GLOBAL VARS
num_encoder_tokens=0; num_decoder_tokens=0; 
input_token_index={}; target_token_index={}
max_encoder_seq_length=0; max_decoder_seq_length=0;
input_characters=set()
target_characters = set()
##########################################################################################################################

def preprocessing_TrainValTest(dataType, datapath, mode, isTrain, en_format):
  print("preprocessing started for"+dataType)
  global max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, num_encoder_tokens, num_decoder_tokens, input_characters, target_characters
  input_texts, target_texts = [], []
  with open(datapath, mode, encoding=en_format) as data:
    lines = data.read().split("\n")
  #print(len(lines))
  prev, __, _= lines[0].split("\t"); flag=1
  for iii, line in enumerate(lines[: len(lines) - 1]):
    target_text, input_text, ignore = line.split("\t")
    if iii>0 and target_text==prev : continue;
    prev=target_text
    #if target_text in target_texts : continue
    start = "\t"; end="\n"
    target_text = start + target_text + end ;  input_text = start + input_text + end
    input_texts.append(input_text) ;   target_texts.append(target_text)
    if isTrain:
      for cc in input_text:
          if cc not in input_characters:
              input_characters.add(cc)
      for cc in target_text:
          if cc not in target_characters:
              target_characters.add(cc)


  if isTrain==1:

    input_characters = sorted(list(input_characters))
    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

    target_characters = sorted(list(target_characters))
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

    num_encoder_tokens = len(input_characters)
    max_encoder_seq_length = max([len(ii) for ii in input_texts])+1

    num_decoder_tokens = len(target_characters)
    max_decoder_seq_length = max([len(ii) for ii in target_texts])+1 

    bookKeep(input_texts)



  encoder_input_data, decoder_input_data, decoder_target_data= eid_did_dtd(input_texts)

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t] =  input_token_index[char]
      encoder_input_data[i, t + 1: ] = input_token_index[end]
      for t, char in enumerate(target_text):
          decoder_input_data[i, t] = target_token_index[char]
          if t > 0:
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1: ] = target_token_index[end]
      decoder_target_data[i, t:, target_token_index[end]] = 1.0
  print("preprocessing finished for"+dataType+"\n")
  return input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
def setPaths():
    aaaaaa="dakshina_dataset_v1.0/hi/lexicons/"
    bbbbbb =  aaaaaa+"hi.translit.sampled.train.tsv"
    cccccc =  aaaaaa+"hi.translit.sampled.dev.tsv"
    dddddd =  aaaaaa+"hi.translit.sampled.test.tsv"
    return aaaaaa, bbbbbb, cccccc, dddddd

In [ ]:
root, train_data_path, val_data_path, test_data_path = setPaths()
input_train_texts, target_train_texts, encoder_input_train_data, decoder_input_train_data, decoder_target_train_data = preprocessing_TrainValTest(" train", train_data_path, mode="r", isTrain=1, en_format="utf-8")
input_val_texts, target_val_texts, encoder_input_val_data, decoder_input_val_data, decoder_target_val_data = preprocessing_TrainValTest(" val" ,val_data_path, mode="r", isTrain=0, en_format="utf-8")
input_test_texts, target_test_texts, encoder_input_test_data, decoder_input_test_data, decoder_target_test_data = preprocessing_TrainValTest(" test" ,test_data_path, mode="r", isTrain=0, en_format="utf-8")

In [ ]:
def shapes():
   xx=keras.Input(shape=(max_encoder_seq_length,))
   yy=keras.Input(shape=(max_decoder_seq_length,))
   return xx, yy

def makeModel(xxx, yyy):
   return keras.Model(xxx, yyy)

In [ ]:
# RNN based seq2seq model

keepCells = {"gru":keras.layers.GRU, "lstm":keras.layers.LSTM, "vanillaRnn":keras.layers.SimpleRNN, "embedd":keras.layers.Embedding, "dense":keras.layers.Dense}

def ReccNeuralNet(en, input_embedd_sz, latent_dim, drop, cell):
   
    global putEncoders, HiddenStatesOfDecoder, putDecoders


    # (i) input layer for character embeddings

    encoder_inputs, decoder_inputs=shapes()

    encoder_layer=keepCells["embedd"](input_dim= num_encoder_tokens, output_dim=input_embedd_sz)
    encoder_embedded = encoder_layer(encoder_inputs)
    decoder_layer=keepCells["embedd"](input_dim = num_decoder_tokens, output_dim=input_embedd_sz )
    decoder_embedded=decoder_layer(decoder_inputs)
    # (i) input layer for character embeddings



    putEncoders, putDecoders, HiddenStatesOfDecoder, en = [], [], [], en-1
    de=en




    # (ii) one encoder RNN which sequentially encodes the input character sequence
    if cell=="vanillaRnn":

        ################################################################################################
        encoder = keepCells[cell](latent_dim , return_state=1, return_sequences=1)
        outputs, state_h = encoder(encoder_embedded)
        HiddenStatesOfDecoder.append([state_h])
        putEncoders.append(encoder)

        for i in range(en):
          en_rnn = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          outputs, state_h = en_rnn(outputs)
          HiddenStatesOfDecoder.append([state_h])
          putEncoders.append(en_rnn)
        ################################################################################################

    elif cell=="lstm":

        ################################################################################################
        encoder = keepCells[cell](latent_dim , return_state=1, return_sequences=1)
        outputs, state_h, state_c = encoder(encoder_embedded)
        HiddenStatesOfDecoder.append([state_h, state_c])
        putEncoders.append(encoder)
        for i in range(en):
          en_lstm =keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          outputs, state_h, state_c = en_lstm(outputs)
          HiddenStatesOfDecoder.append([state_h, state_c])
          putEncoders.append(en_lstm)
        ################################################################################################

    elif cell=="gru":

        ################################################################################################
        encoder = keepCells[cell](latent_dim , return_state=1, return_sequences=1)
        outputs, state_h = encoder(encoder_embedded)
        HiddenStatesOfDecoder.append([state_h])
        putEncoders.append(encoder)
        for i in range(en):
          en_gru = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          outputs, state_h = en_gru(outputs)
          HiddenStatesOfDecoder.append([state_h])
          putEncoders.append(en_gru)
        ################################################################################################


    # (ii) one encoder RNN which sequentially encodes the input character sequence
     



    # (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time

    if cell=="vanillaRnn":

        ################################################################################################
        decoder_rnn = keepCells[cell](latent_dim, return_sequences=1, return_state=1)
        de_outputs, ignore = decoder_rnn(decoder_embedded, initial_state=HiddenStatesOfDecoder[0])
        putDecoders.append(decoder_rnn)
        for i in range(de):
          de_rnn = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          de_outputs, ignore = de_rnn(de_outputs,HiddenStatesOfDecoder[i+1])
          putDecoders.append(de_rnn)
        ################################################################################################

    elif cell=="lstm":

        ################################################################################################
        decoder_lstm = keepCells[cell](latent_dim, return_sequences=1, return_state=1)
        de_outputs, ignore1, ignore2 = decoder_lstm(decoder_embedded, initial_state=HiddenStatesOfDecoder[0])
        putDecoders.append(decoder_lstm)
        for i in range(de):
          de_lstm = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          de_outputs, state_h, state_c = de_lstm(de_outputs, initial_state=HiddenStatesOfDecoder[i+1])
          putDecoders.append(de_lstm)
        ################################################################################################

    elif cell=="gru":

        ################################################################################################
        decoder_gru = keepCells[cell](latent_dim, return_sequences=1, return_state=1)
        de_outputs, ignore = decoder_gru(decoder_embedded, initial_state=HiddenStatesOfDecoder[0])
        putDecoders.append(decoder_gru)
        for i in range(de):
          de_gru = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          de_outputs, ignore = de_gru(de_outputs, HiddenStatesOfDecoder[i+1])
          putDecoders.append(de_gru)
        ################################################################################################

    # (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time


  

    # Dense Layer
    makeDense=keepCells["dense"](num_decoder_tokens, activation="softmax")
    decoder_dense = makeDense
    decoder_outputs = decoder_dense(de_outputs) # None*22*65
    # Dense Layer
    
    #model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model = makeModel([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

In [ ]:
import numpy as np
def summary_architechture(model):
  print(model.summary())
  for i in range (len(model.layers)):
     print(i, model.layers[i])

In [ ]:
def plotModel(mmm):
  return tf.keras.utils.plot_model(mmm , show_shapes = True)

In [ ]:
a,b,c,d,e=2, 128, 256, 0.5, "lstm"
f,g,h,i="rmsprop", 3e-3, 17, 32
model=ReccNeuralNet(a,b,c,d,e)
summary_architechture(model)

In [ ]:
plotModel(model)